# Preprocessing for Curb Data

In [2]:
# Enable autoload for just updated files
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import numpy as np
sys.path.append('../../')   # Add parent directory to Python path
from utils.preprocessing import *
from utils.segmentation import *
from utils.visualization import *

# #P1 Cobblestone Road E1-E3

In [9]:
#Missing value and combined data
#Load the E1 Data and handle missing values
df_one= pd.read_csv('../../data/RoadRoughness/Raw/Cobblestone/P2/E1/HandleBar/Accelerometer/Accelerometer.0.csv')  
df_one


,Date,NTP,GNSS-Time,Acc-X,Acc-Y,Acc-Z
0,2024-11-18T16:15:44.346,2024-11-18 16:15:44.059,-1,-1.200599,3.766602,8.971889
1,2024-11-18T16:15:44.348,2024-11-18 16:15:44.061,-1,-1.029213,3.920341,8.869894
2,2024-11-18T16:15:44.353,2024-11-18 16:15:44.066,-1,-0.984646,3.835994,8.807981
3,2024-11-18T16:15:44.357,2024-11-18 16:15:44.070,-1,-0.934696,3.902395,8.776276
4,2024-11-18T16:15:44.360,2024-11-18 16:15:44.072,-1,-0.953241,3.905685,8.870791
...,...,...,...,...,...,...
6591,2024-11-18T16:16:10.708,2024-11-18 16:16:10.421,-1,1.691726,3.931408,9.413065
6592,2024-11-18T16:16:10.713,2024-11-18 16:16:10.425,-1,1.469193,3.914359,9.513564
6593,2024-11-18T16:16:10.717,2024-11-18 16:16:10.430,-1,1.384248,4.000500,9.229416
6594,2024-11-18T16:16:10.720,2024-11-18 16:16:10.433,-1,1.346860,4.146462,8.867501


In [5]:
plot_accelerometer_data(df_one,None)

In [10]:
def filter_activity_period(df, threshold=1.0):
    """
    Filter the dataframe to keep only the active period where significant acceleration occurs.
    
    Args:
        df: DataFrame with accelerometer data
        threshold: Standard deviation threshold to detect activity
    
    Returns:
        tuple: (filtered DataFrame, start_time)
    """
    # Calculate rolling standard deviation for Acc-Z (seems to show most activity)
    rolling_std = df['Acc-Z'].rolling(window=100).std()
    
    # Find regions where standard deviation is above threshold
    active_regions = rolling_std > threshold
    
    # Find the first and last indices where activity occurs
    start_idx = active_regions.idxmax()
    end_idx = len(df) - active_regions[::-1].idxmax()
    
    # Get the start time using 'ntp' column
    start_time = df.iloc[start_idx]['ntp']
    print(f"Activity start time: {start_time}")
    
    # Filter the dataframe
    filtered_df = df.iloc[start_idx:end_idx].copy()
    
    return filtered_df, start_time

In [11]:
# Update how you call the function
filtered_df_one, start_time = filter_activity_period(df_one)

# Plot the filtered data to verify
plot_accelerometer_data(filtered_df_one, None)

KeyError: 'ntp'